<a href="https://colab.research.google.com/github/MScEcologyAndDataScienceUCL/BIOS0032_AI4Environment/blob/main/4_AI_for_Bioacoustics/AI_for_Bioacoustics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 4 - AI for Bioacoustics

## What we will learn

In this weeks practical we will explore computer audition applications in ecology such as automated animal detection and species classification from audio sensor data. Among other stuff we will:

1. Learn how to visualize audio data.
2. Apply an automated animal detector on a set of audio files.
3. Learn to evaluate the performance of a sound detector.
4. Use a trained neural network to detect animals.
5. Manually extract relevant sound features.
6. Use features to create an automated animal sound identifier.
7. Compare between hand-crafted feature sets with learned features.

If time permits, we will have some time exploring how to train a neural network for audio classification.

## Intro

### Computer audition

**What is Computer audition?**

- Computer audition is the field of research that deals with the automatic
  analysis of audio signals.

- It intersects with many other fields, including machine learning, signal
  processing, and computer vision.

**What does a computer hear**?

- Audio files are a sequence of numbers representing the amplitude of the sound
  wave at a given time.

- The number of samples per second is called the **sampling rate**.

<img alt="audio and sampling rate" width="600" src="https://cdn.shopify.com/s/files/1/1169/2482/files/Sampling_Rate_Cover_image.jpg?v=1654170259"></img>

**What tasks can we do with computer audition?**

Computer audition is used in a wide range of applications, including:

- Speech recognition: Siri, Alexa, Google Assistant
- Music information retrieval: Spotify, Shazam
- Audio classification: What is sounding in this audio?
- Sound event detection: Transcription of audio into a sequence of events.

<img alt="Sound event detection" width="400" src="http://d33wubrfki0l68.cloudfront.net/508a62f305652e6d9af853c65ab33ae9900ff38e/17a88/images/tasks/challenge2016/task3_overview.png"></img>

> Taken from the paper: Mesaros, A., Heittola, T., Diment, A., Elizalde, B., Shah, A., Vincent, E., ... & Virtanen, T. (2017, November). DCASE 2017 challenge setup: Tasks, datasets and baseline system. In DCASE 2017-Workshop on Detection and Classification of Acoustic Scenes and Events.

Recently neural network models have taken over the field of computer audition and are being used to solve many of the above tasks.

### Data collection

**Acoustic sensors** can be used to collect field recordings of animal sounds.

Usually, these sensors are deployed statically in the field for a long periods of time and record sounds continuously. This is called **passive acoustic monitoring**.

<img alt="passive acoustic monitoring" width="400" src="https://wittmann-tours.de/wp-content/uploads/2018/06/AudioMoth.jpg"></img>

Alternatively, recordings are actively directed towards a specific animal species or sound events.

<img alt="active recording" width="400" src="https://s3.amazonaws.com/cdn.freshdesk.com/data/helpdesk/attachments/production/48032687175/original/xjI7Dy3Q9kaCZinr5vf4ksNxQbjK13Yv3A.jpg?1584552543"></img>

> Taken from the Macaulay Library blog post:
> [Sound recording tips](https://support.ebird.org/en/support/solutions/articles/48001064298-sound-recording-tips)

### Acoustics for ecology

The sound at a site is a reflection of the species present in the area and other
environmental factors.

<img alt="composition of acoustic space" width="400" src="https://media.springernature.com/full/springer-static/image/art%3A10.1007%2Fs12304-017-9288-5/MediaObjects/12304_2017_9288_Fig1_HTML.gif?as=webp"></img>

> Taken from the paper: Mullet, T.C., Farina, A. & Gage, S.H. The Acoustic
> Habitat Hypothesis: An Ecoacoustics Perspective on Species Habitat Selection.
> Biosemiotics 10, 319–336 (2017). https://doi.org/10.1007/s12304-017-9288-5

If we could link the sounds to the species, we could use this information to
study and monitor the biodiversity of an area.

Acoustic sensors produce a lot of data, and it is not always easy to analyse.
Can we use computer audition to help us?

In this practical we will explore the task of **animal sound detection** and
**species classification**, using both manual and automated methods.

## Setup Steps

Here we will go through the steps to setup the environment for this practical.

### Make sure to use GPU runtime in Colab. 

Go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

### Mount your Google Drive. 

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Add a shortcut in you drive to this [shared folder](https://drive.google.com/drive/folders/1hbbbsILNBsQghktuj0z_Jq_3iEZQCCbj?usp=share_link).

This will allow you to access the data we will use in this practical.

In [ ]:
%%capture
# Extract data into machine
!unzip /content/drive/MyDrive/BIO0032_AI4Environment/week4_data.zip -d /content/week4_data

### Install and import dependencies. 

Run the following cell to install the required dependencies. This will take a few minutes. You can omit the outputs of this cell.

In [ ]:
%%capture
!sudo apt-get install libfftw3-dev libicu-dev libsndfile1-dev libqt5core5a
!pip install pytadarida git+https://github.com/MScEcologyAndDataScienceUCL/BIOS0032_AI4Environment.git git+https://github.com/mbsantiago/batdetect2 umap-learn

In [ ]:
# @title Import dependencies

import glob
import os
from pathlib import Path
from time import perf_counter

import ipywidgets
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import tensorflow_hub as hub
import xarray as xa
from IPython.display import Audio
from librosa import display
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    classification_report,
    confusion_matrix,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from umap import UMAP

from bios0032utils.bioacoustics import detection, evaluate_detection, plotting
from bios0032utils.bioacoustics.classification import (
    TADARIDA_FEATURES,
    load_bat_call_audio_data,
)

## Part 1: Detecting Animal Sounds

- Most of the time, we are not interested in all the sounds in a recording, but only in the sounds of a specific animal species.

- Acoustic sensors will indiscriminately record all sounds in the environment, including those of animals, wind, rain, etc. Although some recorders can be triggered by a specific sound, this is not always the case.

- Passive acoustic monitoring produces many hours of recordings, and it's hard to identify and explore the sounds of interest.

These are similar problems to the ones we have seen in the previous practicals for camera trap images.

While not as developed as in **computer vision**, there are some tools for automatically **detecting animal sounds** in recordings. Here we will explore a few of them.

But first we need to understand how to visualise and annotate sounds.

### Animal sounds visualisation

- While we can listen to the sounds in a recording, it is often easier to
visualise them.

- This is especially true when we want to compare sounds from
different recordings or navigate quickly without the need to listen.

- We can use waveplots and spectrograms to visualise the sounds in a recording.

Now we will load a dataset of animal recordings provided by [Avisoft](https://www.avisoft.com/animal-sounds/) and visualize them.

In [ ]:
DATA_DIR = Path("data")

In [ ]:
AVISOFT_AUDIO_DIR = DATA_DIR / "avisoft" / "audio"
AVISOFT_METADATA_FILE = DATA_DIR / "avisoft" / "avisoft_metadata.csv"

In [ ]:
# Load the metadata dataframe
avisoft = pd.read_csv(AVISOFT_METADATA_FILE)

In [ ]:
# Print first few rows
avisoft.head()

In [ ]:
# select a random file from the dataset
random_recording = avisoft.sample(n=1).iloc[0]

# read the audio file and import it as a numpy array
wav, samplerate = librosa.load(
    os.path.join(AVISOFT_AUDIO_DIR, random_recording.wav),
    sr=None,
)

# Compute the duration of audio
num_samples = len(wav)  # Number of samples taken by the recorder
duration = num_samples / samplerate

# Get name of animal
animal_name = random_recording.english_name

print(f"File selected = {random_recording.wav}")
print(f"Samplerate = {samplerate} Hz")
print(f"Duration = {duration:.2f} s")
print(f"Species = {animal_name}")

Lets first listen to the audio

In [ ]:
Audio(data=wav, rate=samplerate)

In [ ]:
# Create plot of the waveform
times = np.linspace(0, duration, num_samples)
plt.figure(figsize=(10, 3))
plt.plot(times, wav)
plt.xlabel("time (s)")
plt.title(f"Waveform of {animal_name} sound");

The **waveform** gives us a visual representation of the sound amplitude over time.

However, ff there are multiple simultaneous sounds in the recording, it can be hard to see each individual sound. 

We can use a **spectrogram** to decompose the sound into **frequencies** and visualise them as a 2D image.

In [ ]:
# Compute the spectrogram with the short time fourier transform (STFT)
spectrogram = np.abs(librosa.stft(wav))

# Amplitude is best represented in logarithmic scale (decibels)
db_spectrogram = librosa.amplitude_to_db(spectrogram, ref=np.max)

In [ ]:
# Create plot of spectrogram
num_freq_bins, num_time_bins = db_spectrogram.shape
times = np.linspace(0, duration, num_time_bins)
freqs = np.linspace(0, samplerate / 2, num_freq_bins)

plt.figure(figsize=(10, 4))
plt.pcolormesh(times, freqs, db_spectrogram, cmap="magma")
plt.colorbar()
plt.title(f"Spectrogram of {random_recording.english_name} sound")
plt.xlabel("time (s)")
plt.ylabel("freq (Hz)");

### Exercise 🐘

The sounds produced by animals can be very different from each other. The
transformation used to create the spectrogram, called the **short-time Fourier
transform** (STFT), will highlight different features of the sound depending on
the parameters used.

- Research what the STFT is and how its parameters affect the spectrogram. In
  particular, try to understand the effect of the **window size** and the **hop
  size** or **overlap**.

The **Fourier transform** is a method for breaking a time series into its continuent frequencies. [Click here](https://www.youtube.com/watch?v=spUNpyF58BY) if you are interested in an intuitive and visual explanation of the Fourier transform.

For the **short-time Fourier transform** the audio is broken up into  **windows** (or **chunks** or **frames**), which usually overlap each other. Each **window** is Fourier transformed, and the result is added to a matrix, which records magnitude for each point in time and frequency.

![short time fourier transform](https://www.mdpi.com/applsci/applsci-10-07208/article_deploy/html/images/applsci-10-07208-g001-550.jpg)

In order to compute a **STFT** of a signal you must select a `window_length` (or `n_fft`) and a `hop_length` (or `overlap`) to determine how to break up the signal into **windows**.

* The `hop_size` controls the temporal resolution, or the minimum interval at which you can detect changes in sound. If `hop_length = 128` then any transient sounds of length less than 128 samples will be hard to detect. 

* The `window_length` controls the frequecy resolution. With larger `window_length` it is possible to distinguish between closer frequencies.

* Selecting a high/low value for `window_length` will produce spectrograms with many/few frequency bins.

* Similarly, a high/low value for `hop_length` will produce spectrograms with many/freq time bins.

Here you can visualise sounds from different species and see how the STFT
parameters affect the spectrogram.

In [ ]:
# @title Interactive spectrogram of animal sounds

# @markdown Select the file you wish to visualize. Modify the spectrogram parameters to see its effect on the spectrogram. Change the reproduction speed for interesting effects!

# Select some varied sounds from avisoft dataset
examples = [
    (row.english_name, os.path.join(AVISOFT_AUDIO_DIR, row.wav))
    # select one random recording per taxonomic group
    for row in avisoft.groupby("order").sample(n=1).itertuples()
]

# Create interactive plot
ipywidgets.interact(
    plotting.plot_waveform_with_spectrogram,
    hop_length=(32, 1024, 32),
    n_fft=(32, 2048, 32),
    window=plotting.WINDOW_OPTIONS,
    file=examples,
    cmap=plotting.COLORMAPS,
    speed=[
        ("x1", 1),
        ("x1.5", 1.5),
        ("x2", 2),
        ("x0.5", 0.5),
        ("x0.2", 0.2),
        ("x0.1", 0.1),
    ],
);

- Try changing the parameters and see how they affect sounds from different
  species.

- Can you see that some choice of parameters are good for some species but
  not for others?

Some species have slowly changing frequency, like a Sheep, hence selecting a high `hop_length` would be able to capture its vocalization accurately without few temporal samples.

Other species, such as the Little Grebe, have quickly varying frequencies, and a high `hop_length` would blur the intricacies of its song.

A small `window_length` can be chosen in case identification does not rely on accurate frequency information. For example the Hoopoe call consists of a burst of three rapid pulses at low frequencies. This pattern can be cleary distinguished even with low frequency resolution.

Ofter species will call at similar frequency bands. In such case it's best to select a `window_length` that will produce enough frequency resolution to distinguish between similar calls.

- How do the parameters affect the computation time and resulting image size?

* Larger `window_length` will produce taller spectrograms and slow down computation time.
* Smaller `hop_length` will produce lengthier spectrograms and slow down computation time.

In [ ]:
# take a single file
species, filepath = examples[0]

print(
    f"Will generate multiple spectrograms of {species} sounds. Using the file: {filepath}"
)

# load the audio
wav, sr = librosa.load(filepath, sr=None)

# select multiple choices of window_length and hop_length
window_lengths = np.arange(64, 2048, 64)
hop_lengths = np.arange(32, 1024, 32)

# create list in which to store the resulting computation times
computation_times = []

# iterate over all window_length and hop_length options
for window_length in window_lengths:
    for hop_length in hop_lengths:
        # start counter
        computation_time = perf_counter()

        # compute spectrogram
        spectrogram = librosa.amplitude_to_db(
            np.abs(
                librosa.stft(
                    wav,
                    hop_length=hop_length,
                    n_fft=window_length,
                    window="hann",
                )
            ),
            ref=np.max,
        )

        # end counter
        computation_time = perf_counter() - computation_time

        # store result in computation_times list
        computation_times.append(
            {
                "window_length": window_length,
                "hop_length": hop_length,
                "computation_time": computation_time,
                "spectrogram_size": spectrogram.size,
            }
        )

# convert computation_times list into a pandas dataframe
computation_times = pd.DataFrame(computation_times)

In [ ]:
plt.figure(figsize=(10, 6))
ax = sns.scatterplot(
    data=computation_times,
    x="computation_time",
    y="spectrogram_size",
    size="window_length",
    hue="hop_length",
    sizes=(20, 200),
)
ax.set_xscale("log")
ax.set_yscale("log");

### Detecting sounds

As you can imagine, it is not easy to manually annotate all relevant sounds in a recording.

Look at this recording from a bat detector:

In [ ]:
YUCATAN_METADATA = DATA_DIR / "yucatan" / "yucatan_metadata.csv"

YUCATAN_AUDIO_DIR = DATA_DIR / "yucatan" / "audio"

# Load metadata of dataset of bat recordings from the yucatan peninsula
yucatan = pd.read_csv(YUCATAN_METADATA)

In [ ]:
crowded_recording = os.path.join(YUCATAN_AUDIO_DIR, yucatan.id[1017])

plotting.plot_spectrogram(
    crowded_recording,
    hop_length=128,
    n_fft=512,
    figsize=(14, 4),
)

There are many bat calls in this recording, it would be very time consuming to annotate them all.

In [ ]:
empty_recording = os.path.join(YUCATAN_AUDIO_DIR, yucatan.id[205])
plotting.plot_spectrogram(
    empty_recording,
    hop_length=128,
    n_fft=512,
    figsize=(14, 4),
)

This other recording has a single bat pulse. You still need to review it thoroughly to make sure there are no other sounds of interest.

### Tadarida

Similar to **MegaDetector** for camera traps, there are some tools that automatically detect animal sounds in recordings.

Here we will explore the tool **Tadarida**. Tadarida is a non-ML generic detector that uses a set of hand-crafted features to detect sounds. It is based on the work of:

> Bas, Y., Bas, D. and Julien, J.-F., 2017. Tadarida: A Toolbox for Animal
> Detection on Acoustic Recordings. Journal of Open Research Software, 5(1),
> p.6. DOI: http://doi.org/10.5334/jors.154

We will use tadarida to detect bat calls in a recording.

In [ ]:
detections = detection.run_tadarida_detection([empty_recording, crowded_recording])

In [ ]:
plotting.plot_spectrogram_and_detection(crowded_recording, detections);

In [ ]:
plotting.plot_spectrogram_and_detection(empty_recording, detections);

### Evaluate Detections

The calls of this dataset were manually annotated, so we can compare the detections with the ground truth.

In [ ]:
YUCATAN_ANNOTATIONS = DATA_DIR / "yucatan" / "yucatan_annotations.csv"

In [ ]:
# Load the annotations file
yucatan_annotations = pd.read_csv(YUCATAN_ANNOTATIONS)

Lets first visualize the ground truth annotations.

In [ ]:
plotting.plot_spectrogram_and_detection(empty_recording, yucatan_annotations);

In [ ]:
plotting.plot_spectrogram_and_detection(crowded_recording, yucatan_annotations);

Now we can compare the detections with the ground truth. We can use the
Intersection over Union (IoU) to measure the overlap between the detections and
the ground truth.

<img alt="intersection over union" src="https://upload.wikimedia.org/wikipedia/commons/c/c7/Intersection_over_Union_-_visual_equation.png" width="400"></img>

In [ ]:
# Select the predictions and annotations from the crowded recording
file_detections = detections[
    detections.recording_id == os.path.basename(crowded_recording)
]
file_annotations = yucatan_annotations[
    yucatan_annotations.recording_id == os.path.basename(crowded_recording)
]

# Match the bounding boxes by computing the IoU. Discard all matches with IoU less than 0.5
pred_boxes = evaluate_detection.bboxes_from_annotations(file_detections)
true_boxes = evaluate_detection.bboxes_from_annotations(file_annotations)
matches = evaluate_detection.match_bboxes(true_boxes, pred_boxes, iou_threshold=0.5)

We select all the detections that have an IoU greater than 0.5 and count them as true positives. All the other detections are false positives. Sound events that are not detected are false negatives.

In [ ]:
# total number of annotated sound events
positives = len(file_annotations)

num_predictions = len(file_detections)

# number of matched prediction boxes
true_positives = len(matches)

# number of predicted boxes that were not matched
false_positives = num_predictions - len(matches)

# number of annotated sound events that were not matched
false_negatives = positives - len(matches)

With this information we can compute the precision and recall of the detections.

In [ ]:
# Percentage of predictions that are correct
precision = true_positives / num_predictions

# Percentage of sound events that were detected
recall = true_positives / positives

print(
    f"Tadarida precision={precision:.1%} recall={recall:.1%} on file {crowded_recording}"
)

Lets plot predictions and annotations at the same time.

- red = spurious predicted sound event (false positive)
- green = correct prediction (true positive)
- white = missed sound event (false negative)

In [ ]:
plotting.plot_spectrogram_with_predictions_and_annotations(
    crowded_recording,
    detections,
    yucatan_annotations,
    iou_threshold=0.3,
);

We can also compute the precision/recall on each file in the dataset.

In [ ]:
# load precomputed tadarida detections to save some time
full_tadarida_detections = pd.read_csv(
    DATA_DIR / "yucatan" / "yucatan_tadarida_detections.csv"
)

# compute the precision and recall for each file
td_evaluation = []
for filename in yucatan_annotations.recording_id.unique():
    precision, recall = evaluate_detection.compute_file_precision_recall(
        filename,
        full_tadarida_detections,
        yucatan_annotations,
        iou_threshold=0.5,
    )
    td_evaluation.append({"wav": filename, "precision": precision, "recall": recall})

# store the results in a pandas dataframe
td_evaluation = pd.DataFrame(td_evaluation)

### Exercise 🐋

Using the dataframe with precision and recall of tadarida on each file (`td_evaluation`), calculate: 

- The mean precision and recall across all files.

In [ ]:
# show the first rows of the td_evaluation dataframe
td_evaluation.head()

Notice that some precision and recall values are **NaN**.

Precision is **NaN** when there were no detections made by tadarida.

Recall is **NaN** when there are no true bat sounds in the recording.

In [ ]:
# compute mean precision and recall.
# The pandas mean method will ignore NaNs for mean computation.
mean_precision = td_evaluation.precision.mean()
mean_recall = td_evaluation.recall.mean()

print(f"Mean precision = {mean_precision:.2%}, Mean recall = {mean_recall:.2%}")

- The percentage of files where all bat calls were missed.

As recall is **NaN** when no true bat sounds are in the recording we can ignore these files.

Saying `recall = 0` is equivalent to saying that all bat call were missed. This is because

    recall = true_positives / positives
    
And so for `recall = 0` this needs `true_positives = 0`, equivalently no detection was correct and all bat calls were missed.

In [ ]:
# the length of the td_evaluation dataframe gives the number of evaluated files
total_files = len(td_evaluation)

# remove NaNs from the recall column
non_na_recalls = td_evaluation.recall.dropna()

# check which recalls are 0
recall_is_cero = non_na_recalls == 0

# count the number of True
num_recall_is_cero = recall_is_cero.sum()

# compute the percentage
perc_all_bats_missed = num_recall_is_cero / total_files

print(
    f"Percentage of files where all bat calls were missed: {perc_all_bats_missed:.2%}"
)

- The percentage of files where at least half of the predictions were correct.

We can ignore files were precision is **NaN**, since this occurs when no predictions were made.

Saying `precision >= 0.5` is equivalent to saying that at least half of the predictions are correct.

In [ ]:
# the length of the td_evaluation dataframe gives the number of evaluated files
total_files = len(td_evaluation)

# remove NaNs from the precision column
non_na_precisions = td_evaluation.precision.dropna()

# check which recalls are 0
precision_is_gt_half = non_na_precisions >= 0.5

# count the number of True
num_precision_is_gt_half = precision_is_gt_half.sum()

# compute the percentage
perc_at_least_half_preds_correct = num_precision_is_gt_half / total_files

print(
    f"Percentage of files where at least half of the predictions are correct: {perc_at_least_half_preds_correct:.2%}"
)

Run the full evaluation again but change the `iou_threshold` parameter. What do you observe?

* Increasing the **IoU threshold** makes it harder to find matches (`true_positives`). Recall will necessarily be lowered, as less bat calls can be detected. Precision will also be lowered as less detections can be correct.

* Conversely, a lower **IoU threshold** increases the number of matches (`true_positives`), making both recall and precision greater.

* Selecting an **IoU threshold** is not about optimizing performance, but about choosing a matching criterion. Having a low **IoU** threshold might increase the recall and precision but incurr a cost in precision of the bounding box of each detection.

Lets create a function to do the whole evaluation with `iou_threshold` as a parameter

In [ ]:
def run_whole_evaluation(iou_threshold):
    # compute the precision and recall for each file
    td_evaluation = []
    for filename in yucatan_annotations.recording_id.unique():
        precision, recall = evaluate_detection.compute_file_precision_recall(
            filename,
            full_tadarida_detections,
            yucatan_annotations,
            iou_threshold=iou_threshold,
        )
        td_evaluation.append(
            {"wav": filename, "precision": precision, "recall": recall}
        )

    # store the results in a pandas dataframe
    td_evaluation = pd.DataFrame(td_evaluation)
    return td_evaluation

We create another function to compute evaluation statistics of the previous points:

In [ ]:
def get_evaluation_statistics(td_evaluation):
    # compute mean precision and recall.
    # The pandas mean method will ignore NaNs for mean computation.
    mean_precision = td_evaluation.precision.mean()
    mean_recall = td_evaluation.recall.mean()

    # the length of the td_evaluation dataframe gives the number of evaluated files
    total_files = len(td_evaluation)

    # remove NaNs from the recall column
    non_na_recalls = td_evaluation.recall.dropna()

    # check which recalls are 0
    recall_is_cero = non_na_recalls == 0

    # count the number of True
    num_recall_is_cero = recall_is_cero.sum()

    # compute the percentage
    perc_all_bats_missed = num_recall_is_cero / total_files

    # remove NaNs from the precision column
    non_na_precisions = td_evaluation.precision.dropna()

    # check which recalls are 0
    precision_is_gt_half = non_na_precisions >= 0.5

    # count the number of True
    num_precision_is_gt_half = precision_is_gt_half.sum()

    # compute the percentage
    perc_at_least_half_preds_correct = num_precision_is_gt_half / total_files

    return {
        "mean_precision": mean_precision,
        "mean_recall": mean_recall,
        "perc_all_bats_missed": perc_all_bats_missed,
        "perc_at_least_half_preds_correct": perc_at_least_half_preds_correct,
    }

In [ ]:
results = []

# run evaluation with multiple iou_thresholds
for iou_threshold in tqdm([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
    # evaluate each file with IoU threshold and gather results in dataframe
    evaluation_df = run_whole_evaluation(iou_threshold)

    # compute some evaluation statistics
    stats = get_evaluation_statistics(evaluation_df)

    # specify which threshold was used
    stats["iou_threshold"] = iou_threshold

    # store in results list
    results.append(stats)

# convert results into dataframe
results = pd.DataFrame(results)

In [ ]:
# display dataframe
(
    results.set_index("iou_threshold")
    .style.format(formatter="{:.1%}")
    .format_index("{:.1f}")
)

You can use the following interactive widget to get a better grasp on tadarida's behaviour.

In [ ]:
# @title Tadarida predictions

# @markdown Select a file and an IoU threshold.

example_files = [
    os.path.join(YUCATAN_AUDIO_DIR, row["id"])
    for _, row in yucatan.sample(n=20).iterrows()
]


@ipywidgets.interact(path=example_files, iou_threshold=(0, 1, 0.1))
def plot_results_file_results(path=crowded_recording, iou_threshold=0.5):
    precision, recall = evaluate_detection.compute_file_precision_recall(
        path,
        full_tadarida_detections,
        yucatan_annotations,
        iou_threshold=iou_threshold,
    )

    print(
        f"Tadarida precision={precision:.1%} recall={recall:.1%} on file {crowded_recording}"
    )

    plotting.plot_spectrogram_with_predictions_and_annotations(
        path,
        full_tadarida_detections,
        yucatan_annotations,
        iou_threshold=iou_threshold,
    )

### BatDetect2

As we have seen the performance of Tadarida has room for improvement. We can improve performance by using a specialised machine learning model.

Now we will use **BatDetect2**, a deep learning model for simultaneous
detection and classification of bat calls. Although the model was trained on a
bat calls of UK bats, we can test its detection performance on the dataset of
bats from the Yucatán peninsula.

In [ ]:
%%bash
batdetect2 /content/week4_data/data/yucatan/audio /content/week4_data/data/yucatan/predictions 0.3

The **BatDetect2** model can predict multiple bounding boxes for each recording.
Unlike Tadarida, each bounding box has a **score**, a predicted species and a
confidence score for the species.

We will throw out the predicted species and confidence score, and only use the
bounding box score. The **score** is the probability that the bounding box contains
a bat call.

In [ ]:
# Get all prediction files
files = glob.glob(str(DATA_DIR / "yucatan" / "predictions" / "*.csv"))

# Read each prediction file
batdetect2_predictions = []
for path in files:
    df = pd.read_csv(path).drop(columns=["id", "class", "class_prob"])
    df["recording_id"] = os.path.basename(path)[:-4]
    batdetect2_predictions.append(df)

# And concatenate them into a single dataframe
batdetect2_predictions = pd.concat(batdetect2_predictions)

We can then use the same evaluation procedure as before to compute the
precision and recall, except now we can select detections with a score greater
than some customizable threshold.

In [ ]:
batdetect2_predictions.head()

In [ ]:
score_threshold = 0.3

plotting.plot_spectrogram_with_predictions_and_annotations(
    crowded_recording,
    batdetect2_predictions[batdetect2_predictions.det_prob > score_threshold],
    yucatan_annotations,
    iou_threshold=0.3,
);

In [ ]:
# @title Batdetect2 predictions

# @markdown Select a file and an IoU threshold.

example_files = [
    os.path.join(YUCATAN_AUDIO_DIR, row["id"])
    for _, row in yucatan.sample(n=20).iterrows()
]


@ipywidgets.interact(
    path=example_files,
    iou_threshold=(0, 1, 0.1),
    score_threshold=(0, 1, 0.1),
)
def plot_batdetect2_results_file_results(
    path=crowded_recording,
    iou_threshold=0.5,
    score_threshold=0.3,
):
    confident_detections = batdetect2_predictions[
        batdetect2_predictions.det_prob > score_threshold
    ]

    precision, recall = evaluate_detection.compute_file_precision_recall(
        path,
        confident_detections,
        yucatan_annotations,
        iou_threshold=iou_threshold,
    )

    print(
        f"Batdetect2 precision={precision:.1%} recall={recall:.1%} on file {crowded_recording}"
    )

    plotting.plot_spectrogram_with_predictions_and_annotations(
        path,
        confident_detections,
        yucatan_annotations,
        iou_threshold=iou_threshold,
        linewidth=2,
    )

### Exercise 🐸

Here you will compare the performance of the two detectors.

1. Use the function `evaluate_detection.compute_detection_metrics` to calculate the number
of positive (P), true positives (TP), false positives (FP) and false negatives (FN) for each file.

In [ ]:
# as an example, here is how you use the function
example_file = example_files[0]

confident_detections = batdetect2_predictions[batdetect2_predictions.det_prob > 0.3]

(
    positives,
    true_positives,
    false_positives,
    false_negatives,
) = evaluate_detection.compute_detection_metrics(
    example_file,
    confident_detections,
    yucatan_annotations,
    iou_threshold=0.5,
)

print(
    f"{positives = }, {true_positives = }, {false_positives = }, {false_negatives = }"
)

2. Compute the overall precision and recall for each detector.

In [ ]:
# create list in which to store results
batdetect2_eval = []

# select predictions with confidence score greater that 0.3
confident_detections = batdetect2_predictions[batdetect2_predictions.det_prob > 0.3]

# iterate over each file in the dataset
for filename in yucatan["id"]:
    # compute positives, true_positives, false_positives and false_negatives for file
    (
        positives,
        true_positives,
        false_positives,
        false_negatives,
    ) = evaluate_detection.compute_detection_metrics(
        filename,
        confident_detections,
        yucatan_annotations,
        iou_threshold=0.5,
    )

    batdetect2_eval.append(
        {
            "filename": filename,
            "positives": positives,
            "true_positives": true_positives,
            "false_positives": false_positives,
            "false_negatives": false_negatives,
        }
    )

# convert batdetect2_eval into dataframe
batdetect2_eval = pd.DataFrame(batdetect2_eval)

In [ ]:
# show the first rows to check results
batdetect2_eval.head()

In [ ]:
# will do the same with the tadarida detections
# create list in which to store results
tadarida_eval = []

# iterate over each file in the dataset
for filename in yucatan["id"]:
    # compute positives, true_positives, false_positives and false_negatives for file
    (
        positives,
        true_positives,
        false_positives,
        false_negatives,
    ) = evaluate_detection.compute_detection_metrics(
        filename,
        full_tadarida_detections,
        yucatan_annotations,
        iou_threshold=0.5,
    )

    tadarida_eval.append(
        {
            "filename": filename,
            "positives": positives,
            "true_positives": true_positives,
            "false_positives": false_positives,
            "false_negatives": false_negatives,
        }
    )

# convert batdetect2_eval into dataframe
tadarida_eval = pd.DataFrame(tadarida_eval)

2. Compute the overall precision and recall for each detector.

Precision is defined as

    precision = true_positives / (true_positives + false_positives)
    
and recall as

    recall = true_positives / positives
    
We can compute overall precision and recall by obtaining the total number of `true_positives`, `false_positives` and `positives`.

In [ ]:
# compute batdetec2 precision
batdetect2_precision = batdetect2_eval.true_positives.sum() / (
    batdetect2_eval.true_positives.sum() + batdetect2_eval.false_positives.sum()
)

# compute batdetect2 recall
batdetect2_recall = (
    batdetect2_eval.true_positives.sum() / batdetect2_eval.positives.sum()
)

# computetadarida precision
tadarida_precision = tadarida_eval.true_positives.sum() / (
    tadarida_eval.true_positives.sum() + tadarida_eval.false_positives.sum()
)

# compute tadarida recall
tadarida_recall = tadarida_eval.true_positives.sum() / tadarida_eval.positives.sum()

print(
    f"BatDetect2 -- Precision = {batdetect2_precision:.1%}  Recall = {batdetect2_recall:.1%}"
)
print(
    f"Tadarida   -- Precision = {tadarida_precision:.1%}  Recall = {tadarida_recall:.1%}"
)

3. Which detector performs better?

BatDetect2

4. How does this change if you change the score threshold?

Generally,

* A higher score threshold will result in higher precision and lower recall.
* A lower score threshold will result in lower precision and higher recall.

Lets try out a range of score threshold values and plot the precision-recall for each.

In [ ]:
# create a function to compute precision recall of batdetect2
def compute_batdetect2_precision_recall(score):
    # create list in which to store results
    results = []

    # select predictions with confidence score greater that score
    confident_detections = batdetect2_predictions[
        batdetect2_predictions.det_prob > score
    ]

    # iterate over each file in the dataset
    for filename in yucatan["id"]:
        # compute positives, true_positives, false_positives and false_negatives for file
        (
            positives,
            true_positives,
            false_positives,
            false_negatives,
        ) = evaluate_detection.compute_detection_metrics(
            filename,
            confident_detections,
            yucatan_annotations,
            iou_threshold=0.5,  # note we are using a 50% IoU threshold
        )

        results.append(
            {
                "filename": filename,
                "positives": positives,
                "true_positives": true_positives,
                "false_positives": false_positives,
                "false_negatives": false_negatives,
            }
        )

    # convert results into dataframe
    results = pd.DataFrame(results)

    total_true_positives = results.true_positives.sum()
    total_false_negatives = results.false_positives.sum()
    total_positives = results.positives.sum()
    total_detections = total_true_positives + total_false_negatives

    # compute precision
    precision = (
        np.nan if total_detections == 0 else total_true_positives / total_detections
    )

    # compute recall
    recall = np.nan if total_positives == 0 else total_true_positives / total_positives

    return precision, recall

In [ ]:
precision_recalls = []

# iterate over several score threshold values
for score in tqdm(np.linspace(0.1, 0.9, 9)):
    # compute precision recall for this score threshold value
    precision, recall = compute_batdetect2_precision_recall(score)

    # add to result list
    precision_recalls.append({"score": score, "precision": precision, "recall": recall})

# convert to dataframe
precision_recalls = pd.DataFrame(precision_recalls)

In [ ]:
sns.lineplot(
    data=precision_recalls,
    x="precision",
    y="recall",
    color="black",
)
sns.scatterplot(
    data=precision_recalls,
    x="precision",
    y="recall",
    hue="score",
    s=200,
)
# set x and y axes limits to 0-1
plt.xlim(0, 1)
plt.ylim(0, 1);

## Part 2: Identifying Sounds

- In the previous section we saw how to detect sounds in a recording. But we still need to identify the species that produced the sound.

- Generally, classification is more challenging than detection, as the sounds produced by different species can be very similar (**interspecific overlap**).

- A single species can have flexible vocalisations, think humans or mimic birds such as starling (**intraspecific variation**).

- Bioacoustic data presents similar challenges to the camera trap datasets as recordings can be:
    - **Ocluded** (Simultaneous sounds)
    - **Appear in varying ambient condition**s (rain/wind/thunder)
    - **Partial** (Only captured half the sound)
    - **Noisy** (Saturation and faulty sensor)
    - **Quiet or very loud** (depending on animal size, distance, environment)

For the rest of this notebook we will focus on **10** bat species present in the Yucatán dataset.

In [ ]:
SPECIES = [
    "Mormoops megalophylla",
    "Myotis keaysi",
    "Saccopteryx bilineata",
    "Pteronotus davyi",
    "Pteronotus parnellii",
    "Lasiurus ega",
    "Pteropteryx macrotis",
    "Eumops underwoodi",
    "Rhogeessa aeneus",
    "Eptesicus furinalis",
]

In [ ]:
classification_df = yucatan_annotations[yucatan_annotations["class"].isin(SPECIES)]

### Bat call features

- Previous research on bat call identification was based on hand-crafted features of the bat calls.

- Measuring call features used to be a manual process.

<img src="https://www.elekon.ch/batexplorer2/doc/_images/CallParams.png" alt="call parameters" width="400"/>

> Image taken from the [BatExplorer 2.1 user guide](https://www.elekon.ch/batexplorer2/doc/batcall_params.html).

**Peak frequency [kHz]:**

>    The frequency at which the call is loudest (peak in the spectrum display), aka frequency of maximum energy (FME) or main frequency.
>    Most important parameter for bat classification because it can easily be measured and is often typical for a certain species or group of species.
>    The standard deviation of the peak frequency allows the detection of alternating calling species.
    
**Max frequency [kHz]**

>    The maximum frequency of the call. Often this is equal to the start frequency, for Rhinolophidae typically equal to the peak frequency.
    
**Min frequency [kHz]**

>    The minimum frequency of the call. Often this is equal to the end frequency, for hockey stick calls (e.g. Pipistrelle) it might be lower than the end frequency.
    
**BW Peak2Min [kHz]**

>    Bandwidth Peak2Min = Peak frequency - Min frequency
>    Often used to distinguish Myotis and Pipistrelle calls, Myotis mostly have higher bandwidth.
    
**Call length [ms]**

>    Time period of call start to call end in ms. Can be measured most accurately in the oscillogram (wave rise to wave drop).
>    Search calls from European bats are usually between one and up to approximately 30 ms (horseshoe bats up to 80 ms).
    
**Call distance [ms]**

>    Time period between two consecutive calls in ms. Can be measured most accurately in the oscillogram (wave rise call A to wave rise call B).
>    Often this parameter is not very significant since most bat species have irregular rhythms. But it can be an indicator for behavior.
>    Search calls from European bats usually have distances of about 30 to 300 ms, sometimes even longer.


### Exercise 🐦

* Explore 1 call per species.
* Measure peak frequency, max frequency, min frequency and call length.
* Store the measurements in a pandas dataframe.
* Scatterplot

In [ ]:
to_annotate = classification_df.groupby("class").sample(n=1)


@ipywidgets.interact(
    index=[
        (f"{r['class']}_{index}", i)
        for i, (index, r) in enumerate(to_annotate.iterrows())
    ]
)
def manually_extract_features_from_spectrogram(index=0):
    row = to_annotate.iloc[index]
    return plotting.plot_spectrogram_with_plotly(
        path=os.path.join(YUCATAN_AUDIO_DIR, row["recording_id"]),
        start_time=row["start_time"],
        end_time=row["end_time"],
        low_freq=row["low_freq"],
        high_freq=row["high_freq"],
    ).show()

In [ ]:
# list of measurements of bat call features.
# probably will be different to yours as random calls are selected.
measurements = pd.DataFrame(
    [
        {
            "species": "Saccopteryx bilineata",
            "start_time": 0.0438,
            "end_time": 0.0517,
            "peak_freq": 46511,
            "low_freq": 43066,
            "high_freq": 48234,
        },
        {
            "species": "Rhogeessa aeneus",
            "start_time": 0.0447,
            "end_time": 0.0508,
            "peak_freq": 46511,
            "low_freq": 43066,
            "high_freq": 65460,
        },
        {
            "species": "Pteropteryx macrotis",
            "start_time": 0.04505,
            "end_time": 0.0502,
            "peak_freq": 41343,
            "low_freq": 39621,
            "high_freq": 42205,
        },
        {
            "species": "Pteronotus parnellii",
            "start_time": 0.0401,
            "end_time": 0.05523,
            "peak_freq": 63738,
            "low_freq": 62000,
            "high_freq": 64599,
        },
        {
            "species": "Pteronotus davyi",
            "start_time": 0.0459,
            "end_time": 0.0523,
            "peak_freq": 65460,
            "low_freq": 55125,
            "high_freq": 69767,
        },
        {
            "species": "Myotis keaysi",
            "start_time": 0.0468,
            "end_time": 0.0497,
            "peak_freq": 61154,
            "low_freq": 60292,
            "high_freq": 85271,
        },
        {
            "species": "Mormoops megalophylla",
            "start_time": 0.043,
            "end_time": 0.05,
            "peak_freq": 52541,
            "low_freq": 48234,
            "high_freq": 53402,
        },
        {
            "species": "Eptesicus furinalis",
            "start_time": 0.04476,
            "end_time": 0.05,
            "peak_freq": 34453,
            "low_freq": 33591,
            "high_freq": 39621,
        },
        {
            "species": "Eumops underwoodi",
            "start_time": 0.0427,
            "end_time": 0.0537,
            "peak_freq": 27562,
            "low_freq": 24978,
            "high_freq": 31007,
        },
        {
            "species": "Lasiurus ega",
            "start_time": 0.0412,
            "end_time": 0.0566,
            "peak_freq": 27562,
            "low_freq": 24978,
            "high_freq": 31869,
        },
    ]
)

In [ ]:
# create call length column
measurements["call_length"] = measurements["end_time"] - measurements["start_time"]

In [ ]:
# make a scatterplot of call_length vs peak frequency
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=measurements,
    x="call_length",
    y="peak_freq",
    hue="species",
);

### Tadarida automated features

- Tadarida extracts a large set of features from each detected sound event.

- It is possible to build a pipeline for automated species identification using of automatic feature extraction process.

- First we detect the sounds in the recording, then we extract the features, and
  finally we classify the sounds.

- The classification is done using the extracted features a classifier
  algorithm, like Random Forest classifier.

In [ ]:
# here we load tadarida extracted features
tadarida_features = pd.read_csv(DATA_DIR / "yucatan" / "yucatan_tadarida_features.csv")

# the dataframe contains the class of each detected pulse and
# a suite of automatically extracted parameters
species = tadarida_features["Class"]

print(f"Number of features = {len(TADARIDA_FEATURES)}")
features = tadarida_features[TADARIDA_FEATURES]

In [ ]:
# Now we will split the data into train a test
X_train, X_test, y_train, y_test = train_test_split(
    features,
    species,
    test_size=0.3,
    stratify=species,  # notice we are using the stratified argument
)

In [ ]:
# Lets see the number of examples per species in each dataset
# Train dataset
y_train.value_counts().plot.bar();

In [ ]:
# Test dataset
y_test.value_counts().plot.bar();

Now we are ready to train a classifier to identify the species of the bat calls.

We will use a **Random Forest Classifier** from `scikit-learn`.

In [ ]:
# build a random forest model
rf_model = RandomForestClassifier()

# fit to training data
rf_model.fit(X_train, y_train)

We can evaluate the performance of the classifier in a similar fashion to previous practicals.

There is a useful `classification_report` function from `sklearn.metrics` to compute the precision, recall and F1 score for each species.

In [ ]:
# predict with rf model on evaluation data
y_pred = rf_model.predict(X_test)

print(classification_report(y_true=y_test, y_pred=y_pred))

### Exercise 🦗

In this exercise you will analyze the model's performance in depth.

1. Plot the confusion matrix using the `sklearn.metrics.ConfusionMatrixDisplay` from scikit learn.

If you are confused it is always helpful to look at the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html) and provided examples.

In [ ]:
# compute the confusion matrix
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)

# plot the confusion matrix using the ConfusionMatrixDisplay
ConfusionMatrixDisplay(cm, display_labels=rf_model.classes_).plot(
    xticks_rotation="vertical"
);

In [ ]:
# will compute the confusion matrix again, but normalize each row
cm = confusion_matrix(y_true=y_test, y_pred=y_pred, normalize="true")

_, ax = plt.subplots(figsize=(8, 8))

# plot the confusion matrix using the ConfusionMatrixDisplay
ConfusionMatrixDisplay(cm, display_labels=rf_model.classes_).plot(
    xticks_rotation="vertical",
    values_format=".0%",
    ax=ax,
);

2. Which are the worst performing species? Can you see why? You might need to go back and see some more examples of bat calls.

**Caveat**: Answers are likely to be different each run, as dataset splits and random forest training are random.

* *Lasiurus intermedius* and *Eptesicus furinalis* were the worst performing species.

This could be explained by:

1. Both species have few training examples:

    a. The training examples might not cover the whole range of call variation. Test calls could contain call types not seen during training. This could be fixed by collecting more data, or generating synthetic data.
    
    b. In general in unbalanced datasets the classes with least amount of examples will suffer, as the training algorithms bias model parameters to benefit the most frequent classes. This could be fixed by subsampling overrepresented classes or oversampling underrepresented classes.
    
    c. Each species has around 70-80 calls in the training dataset. However, we are using a set of 143 features, meaning that any model with sufficient flexibility can fully overfit to the training data. This could be fixed by constraining model flexiblity (modify RF params, or select another model type) or using less features.
    
2. There is substantial overlap between echolocation calls of different species:

    a. This could be due to errors in labelling. Errors in the annotation process are common and should be expected. This could be fixed by conducting a review process in which suspicious annotations are verified or corrected. An annotation can be flagged as suspicious if its features lies far from the usual distribution (outlier detection), or if the model is confidently wrong about its class (hard example mining).
    
    b. Bat echolocation calls are are used to navigate and detect pray. If habitual flying space and/or prey are similar for two species they will tend to echolocate in a similar fashion. When this is the case using a different classification scheme for bats, such as one based on ecological traits, might be better suited for the task of acoustic identification.
    
    c. Sound production is also constrained by morphology. Closely related species will tend to have similar sound production capabilities and thus similar echolocation calls. However, this is not always the case as echolocation call is heavily constrained by ecology (as mentioned in the previous point). A potential workaround is to classify to a higher taxonomic level such as genus or family.
    
    d. Selected call features cannot discern between species. It could be the case that there are subtle cues in the audio that aid identification but haven't been measured. In this case either more features should be designed and measured, or adopting an approach of automated feature extraction (such as deep learning approaches) could help find a better feature set for acoustic identification.

Here we will illustrate both points empirically with some plots: 

1. Plot the relation between the precision and recall of a species and the number of training examples.

2. Plot measured features per species to study overlap.

In [ ]:
# create the classification report and convert it to a pandas dataframe
report = (
    pd.DataFrame(
        classification_report(
            y_true=y_test,
            y_pred=y_pred,
            output_dict=True,
        )
    )
    .drop(  # remove unwanted columns
        columns=[
            "accuracy",
            "macro avg",
            "weighted avg",
        ]
    )
    .T.rename(columns={"support": "test_samples"})
    .astype({"test_samples": int})
)

# compute the number of samples per species
train_samples = y_train.value_counts()

# add column with number of train samples
report["train_samples"] = train_samples

report = report.reset_index(names="species")

In [ ]:
# print the report table
report

In [ ]:
# plot precision vs recall, use number of train samples to set marker size
plt.figure(figsize=(8, 8))
sns.scatterplot(
    data=report,
    x="precision",
    y="recall",
    style="species",
    hue="train_samples",
    size="train_samples",
    sizes=(20, 400),
    palette="viridis",
)
# set x and y limits
plt.xlim(0, 1)
plt.ylim(0, 1);

To plot overlap between call features we will:

1. Project the call features into 3-dimension using UMAP. We will only use the test data.
2. Create an interactive 3D-scatterplot with the projected features. Each point will be colored and styled depending on the species.

By clicking on a species in the plot legend you can toggle its visibility. 

In [ ]:
# Standardize each column in the dataset. Standardized is mean=0 variance=1.
X_test_scaled = StandardScaler().fit_transform(X_test)

# Project into three dimensional space with UMAP
X, Y, Z = UMAP(n_components=3).fit_transform(X_test_scaled).T

# Use plotly express to create interactive 3D scatterplot
px.scatter_3d(
    x=X,
    y=Y,
    z=Z,
    color=y_test,
    symbol=y_test,
    height=600,
    opacity=0.6,
)

### Universal feature set

- Is Tadarida's feature set optimal for bat call identification?

- There is a huge amount of information in the recorded audio that we are throwing away by using hand-crafted features.

- Can we learn a better feature set from the data?

Here instead of relying on hand-crafted features we will use acoustic feature extractor called [Yamnet](https://github.com/tensorflow/models/tree/master/research/audioset/yamnet).

Yamnet was trained on [AudioSet](http://research.google.com/audioset/) a massive dataset of YouTube recordings with more than 5.8 thousand hours of audio.

It was trained to classify sounds clips into 527 different classes. The features it learned to extract are thus useful to distinguish and identify a large variety of sounds.

<img src="http://research.google.com/audioset/resources/histogram.svg" alt="audioset dataset" width="400"/>

Audioset does not contain ultrasonic recordings, and thus is devoid of bat sounds. However, we expect the learnt features to be sufficiently general that it can help identify bat calls.

First we need to download the model. Thankfully the model is available in [Tensorflow Hub](https://tfhub.dev/), a repository of pre-trained models.

In [ ]:
# Here we will load the model using tensorflow_hub
yamnet = hub.load("https://tfhub.dev/google/yamnet/1")

Next we will load all the audio bits from the dataset and extract the features using the model.

The yamnet was originally trained with 0.96 second audio clips sampled at 16kHz. 

All our recordings have a samplerate of 441kHz so we will only feed the model with 34ms of audio at a time. 

We will center each audio clip on the bat calls.

In [ ]:
# Here we will load the information of where each bat call starts and ends
audio_clips = pd.read_csv(DATA_DIR / "yucatan" / "yucatan_species_clips.csv")

In [ ]:
audios = []
for _, row in audio_clips.iterrows():
    wav = load_bat_call_audio_data(
        str(YUCATAN_AUDIO_DIR / row.recording_id),
        row.start_time,
        row.end_time,
    )
    audios.append(wav)

In [ ]:
yamnet_features = []
for wav in tqdm(audios):
    scores, feats, spectrogram = yamnet(wav)
    yamnet_features.append(feats.numpy().squeeze())
yamnet_features = np.array(yamnet_features)

Notice the yamnet model returns a set of 1024 features for each audio bit.

In [ ]:
print(yamnet_features.shape)

We can visualize the features using the dimensionality reduction technique
explore in the previous practicals, [UMAP](https://umap-learn.readthedocs.io/en/latest/).

We will compare tadarida's features with the features extracted by yamnet.

In [ ]:
umap_yamnet = UMAP().fit_transform(StandardScaler().fit_transform(yamnet_features))

In [ ]:
X, Y = umap_yamnet.T
plt.figure(figsize=(10, 10))
sns.scatterplot(x=X, y=Y, hue=audio_clips["class"]);

In [ ]:
X_tadarida = tadarida_features[TADARIDA_FEATURES]
y_tadarida = tadarida_features["Class"]

umap_tadarida = UMAP().fit_transform(StandardScaler().fit_transform(X_tadarida))
X, Y = umap_tadarida.T
plt.figure(figsize=(10, 10))
sns.scatterplot(x=X, y=Y, hue=y_tadarida);

Its hard to evaluate between the two feature sets by just these plots. In the next exercise we will train a classifier using the features extracted by Yamnet.

### Exercise 🐝

Here you will build a random forest classifier using the features extracted by Yamnet.

1. Split the data into train and test. You can copy the code from the previous exercise.

In [ ]:
# split the data into train and test
X_train_yamnet, X_test_yamnet, y_train_yamnet, y_test_yamnet = train_test_split(
    yamnet_features,
    audio_clips["class"],
    test_size=0.3,
    stratify=audio_clips["class"],  # notice we are using the stratified argument
)

2. Train a random forest classifier using the training data.

In [ ]:
# build a random forest model
rf_model_2 = RandomForestClassifier()

# fit to training data
rf_model_2.fit(X_train_yamnet, y_train_yamnet);

3. Evaluate the performance using the classification_report function.

In [ ]:
# predict on the test set
y_pred_yamnet = rf_model_2.predict(X_test_yamnet)

# evaluate using the classification_report
print(classification_report(y_true=y_test_yamnet, y_pred=y_pred_yamnet))

4. Plot the confusion matrix.

In [ ]:
# will compute the confusion matrix again, but normalize each row
cm = confusion_matrix(y_true=y_test_yamnet, y_pred=y_pred_yamnet, normalize="true")

_, ax = plt.subplots(figsize=(8, 8))

# plot the confusion matrix using the ConfusionMatrixDisplay
ConfusionMatrixDisplay(cm, display_labels=rf_model_2.classes_).plot(
    xticks_rotation="vertical",
    values_format=".0%",
    ax=ax,
);

5. Was this better? What are the advantages and disadvantages of using a pre-trained model?

**No**, model performance dropped for every species.

Advantages:

1. No feature selection stage.
2. If the dataset on which it was trained closely resembles the target dataset then it will provide a substantial performance boost (as seen in the previous Lab).
3. Can be finetuned to current dataset.

Disadvantages:

1. Not necessarily better than a specialised approach. Tadarida features were extracted from several decades of acoustic research on bat echolocation. 
2. Features are **very** hard to interpret.